# Note
- [Training script](https://www.kaggle.com/code/shelterw/training-llama3-8b-4-bit-qlora-sft)

# Import

In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links /kaggle/input/llm-pip-2024727
!pip install -q -U transformers --no-index --find-links /kaggle/input/llm-pip-2024727
!pip install -q -U tokenizers --no-index --find-links /kaggle/input/llm-pip-2024727
!pip install -q -U peft --no-index --find-links /kaggle/input/llm-pip-2024727

In [ ]:
import time
import torch
import sklearn
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.cuda.amp import autocast
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from threading import Thread
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType 
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import BitsAndBytesConfig, LlamaForCausalLM, LlamaModel, LlamaPreTrainedModel
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from transformers import set_seed
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)
assert torch.cuda.device_count() == 2, "Sorry - multi-GPU required!"

In [ ]:
MODEL_NAME = '/kaggle/input/llama-3-1-8b-instruct-bnb-4bit'
WEIGHTS_PATH = '/kaggle/input/sft-llama3-1-lora-9174'
MAX_LENGTH = 2400
BATCH_SIZE = 2
DEVICE = torch.device("cuda")    

# Prepare Data 

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
def tokenize(example, tokenizer):
    prompts = tokenizer(eval(example['prompt'], {"null": ""}), add_special_tokens=False)["input_ids"]
    responses_a = tokenizer(eval(example['response_a'], {"null": ""}), add_special_tokens=False)["input_ids"]
    responses_b = tokenizer(eval(example['response_b'], {"null": ""}), add_special_tokens=False)["input_ids"]
    assert len(prompts) == len(responses_a) == len(responses_b), "Lengths of prompts, responses_a, and responses_b do not match"
    prompts, responses_a, responses_b = prompts[::-1], responses_a[::-1], responses_b[::-1]
    prompt, response_a, response_b = [], [], []
    p_len, a_len, b_len = 0, 0, 0
    for p, a, b in zip(prompts, responses_a, responses_b):
        prompt.append(p)
        response_a.append(a)
        response_b.append(b)
        p_len += len(p)
        a_len += len(a)
        a_len += len(b)
        if p_len+a_len+b_len > MAX_LENGTH:
            break
    prompt = [item for sublist in reversed(prompt) for item in sublist]
    response_a = [item for sublist in reversed(response_a) for item in sublist]
    response_b = [item for sublist in reversed(response_b) for item in sublist]
    p_a_b_len = len(prompt) + len(response_a) + len(response_b)
    cut_len = p_a_b_len - MAX_LENGTH
    if cut_len>0:
        prompt = prompt[:-int(len(prompt)/p_a_b_len*cut_len)]
        response_a = response_a[:-int(len(response_a)/p_a_b_len*cut_len)]
        response_b = response_b[:-int(len(response_b)/p_a_b_len*cut_len)]
    prompt = tokenizer('<prompt>: ', add_special_tokens=False)["input_ids"] + prompt
    response_a = tokenizer('\n\n<response_a>: ', add_special_tokens=False)["input_ids"] + response_a
    response_b = tokenizer('\n\n<response_b>: ', add_special_tokens=False)["input_ids"] + response_b
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]
    label_token_id = [128250]
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + label_token_id + [tokenizer.eos_token_id]
    attention_mask = len(input_ids)*[1]
    labels = [-100]* len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + label_token_id + [tokenizer.eos_token_id]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# Tokenize

In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(WEIGHTS_PATH)
LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]
def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        # remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer},
    )
    return tokenized_datasets
test_ds = load_data(test, tokenizer)
test_ds

In [ ]:
data = test_ds.to_pandas()
data["max_len"] = data["input_ids"].apply(len)
data[:3]

In [ ]:
data['input_ids'][0]

In [ ]:
print(tokenizer.decode(data["input_ids"][0]))

# Load model 
We load 1 model on each gpu.  

In [ ]:
class Llama3ForSFT(LlamaPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.post_init()

    def forward(
        self,
        input_ids= None,
        attention_mask= None,
        position_ids = None,
        past_key_values= None,
        inputs_embeds= None,
        labels= None,
        use_cache= None,
        output_attentions= None,
        output_hidden_states = None,
        return_dict= None,
        cache_position = None,
    ):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]
        if self.config.pretraining_tp > 1:
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)
        else:
            logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)

            fake_label_tokens_ids = torch.tensor([128250],device=shift_labels.device)
            label_tokens_ids = torch.tensor(LABEL_IDS,device=shift_labels.device)
#             index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
#             true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]
#             true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)
            true_logits = shift_logits[torch.isin(shift_labels, fake_label_tokens_ids)][:,label_tokens_ids]
#             loss = loss_fct(true_logits, true_labels)

        return CausalLMOutputWithPast(
            loss=loss,
            logits=true_logits,
        )

In [ ]:
# Load base model on GPU 0
device0 = torch.device('cuda:0')
base_model_0 = Llama3ForSFT.from_pretrained(
    MODEL_NAME,
    use_cache=False,
    device_map='cuda:0',
)
# Load base model on GPU 1
device1 = torch.device('cuda:1')
base_model_1 = Llama3ForSFT.from_pretrained(
    MODEL_NAME,
    use_cache=False,
    device_map='cuda:1',
)

# Load weights 

In [ ]:
# Get peft
model_0 = PeftModel.from_pretrained(base_model_0, model_id=WEIGHTS_PATH).to(device0) 
model_0.eval()

model_1 = PeftModel.from_pretrained(base_model_1, model_id=WEIGHTS_PATH).to(device1)
model_1.eval()

# Inference


In [ ]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=BATCH_SIZE, max_length=MAX_LENGTH):
    a_win, b_win, tie = [], [], []

    model.eval()
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        labels = tmp["labels"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        pad_labels=[]
        for label in labels:
            label = list(label) + [tokenizer.pad_token_id]*(input_ids[0].shape[0]-label.shape[0])
            pad_labels.append(label)
        labels = torch.tensor(pad_labels).to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        proba = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    df['winner_model_a'] = a_win
    df['winner_model_b'] = b_win
    df['winner_tie'] = tie
    return df

In [ ]:
st = time.time()

data = data.sort_values("max_len", ascending=False)
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device0, device1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)